In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)

import wave as w
import constants as c
import material as m

# Example usage of simulated materials
This notebook is ment as an reference on how to use `Spectrum` and `Material` classes,
and for visual test of the implementation.

### Generate an example spectrum

In [ ]:
s = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
# s = w.DeltaPlanarWave(wavelength=c.GREEN)
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
s.delay(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
s.plot(ax=ax1, spectrum_axis=ax1.twinx(), wavelength=True)
s.plot_amplitude(z, ax=ax2)
ax2.legend()
fig.tight_layout() 
plt.show()

### Compare different materials

#### Analitical vs constant vs matrix theory

In [ ]:
air = m.EmptySpace(z=z, name="analytic")
common_n = 1
fixed_dielectric = m.ConstantMaterial(z=z, n0=common_n, name="phase shift")
dielectric = m.SimpleDielectric(z=z, n0=1, name="transfer matrices")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

air.plot(ax1, imaginary_axis=ax1.twinx())
fixed_dielectric.plot(ax2, imaginary_axis=ax2.twinx())
dielectric.plot(ax3, imaginary_axis=ax3.twinx())
fig.tight_layout() 
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

air.record(s,s)
air.plot_recent_energy(ax1, label="air")
dielectric.record(s,s)
dielectric.plot_recent_energy(ax1, label="matrix theory")
fixed_dielectric.record(s,s)
fixed_dielectric.plot_recent_energy(ax1, label="correlation")
ax1.legend()

air.transmit(s).plot(ax2)
dielectric.transmit(s).plot(ax2, spectrum_axis=ax2.twinx())
fixed_dielectric.transmit(s).plot(ax2, label="reflected")
plt.show()

### Layered material

In [ ]:
z_layered = np.linspace(-2*c.MICRO, 2*c.MICRO, 1000)
layered = m.LayeredMaterial(z=z_layered, n_layers=20, layer_width=80*c.NANO, n0=1.5, n1=2.0)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
layered.shade_plot(ax1, alpha=0.2)
layered.plot(ax1, imaginary_axis=ax1.twinx())
layered.shade_plot(ax2, alpha=0.2)
layered.reflect(s).plot(ax3, label="reflected", wavelength=True, spectrum_axis=ax3.twinx())
layered.record(s, s)
layered.plot_recent_energy(ax2)
fig.tight_layout() 
plt.savefig("layered2.pdf")
plt.show()

### Composite Material

In [ ]:
z_layered = np.linspace(-2*c.MICRO, 2*c.MICRO, 1000)
z_air = np.linspace(-0.4*c.MICRO, 0, 100)
z_clean = np.linspace(-1.6*c.MICRO, 0, 400)
air = m.EmptySpace(z=z_air)
clean = m.SimpleDielectric(z=z_clean, n0=1.5)
layered = m.LayeredMaterial(z=z_layered, n_layers=20, layer_width=80*c.NANO, n0=1.5, n1=2.0)
composite = m.CompositeMaterial([air, layered, clean, air])

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
composite.shade_plot(ax1, alpha=0.2)
composite.plot(ax1, imaginary_axis=ax1.twinx())
composite.shade_plot(ax2, alpha=0.2)
composite.reflect(s).plot(ax3, label="reflected", wavelength=True, spectrum_axis=ax3.twinx())
composite.record(s, s)
composite.plot_recent_energy(ax2)
fig.tight_layout() 
plt.savefig("layered2.pdf")
plt.show()

### An example recording

In [ ]:
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
s0 = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
dielectric.record(s0, s0)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
dielectric.plot_recent_energy(ax1)
ax1.set_title("interference")
dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
dielectric.reflect(s0).plot(ax3, spectrum_axis=ax3.twinx())
plt.tight_layout()
plt.show()

### Random phase example

In [ ]:
steps = 20
colors = cycler('color', plt.get_cmap('viridis')(np.linspace(0, 1, steps+1)))
colors = colors.by_key()['color']

In [ ]:
dielectric = m.SimpleDielectric(z=z, n0=1)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
ax2twin = ax2.twinx()

for pulse, color in enumerate(colors):
    phase = np.random.uniform(0, np.pi)
    s2 = s0 * np.exp(1j*phase)
    dielectric.plot(ax=ax2, imaginary_axis=ax2twin, alpha=0.1, color=color)
    dielectric.record(s0, s2)
    ax1.plot(dielectric.z[:-1], dielectric.recent_energy[:-1], label="pulse {}".format(pulse + 1), alpha=0.5, c=color)
    ax1.set_title("interference patterns")
    ax1.legend()
    dielectric.reflect(s0).plot(ax3, spectrum_axis=ax4, alpha=0.5, color=color)
    ax3.set_title("reflected")
    ax4.set_title("reflected")
        
plt.tight_layout()        
plt.show()

### Sweeping phase example

In [ ]:
dielectric = m.SimpleDielectric(z=z, n0=1)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
ax2twin = ax2.twinx()

for pulse, color in enumerate(colors):
    phase = np.pi*pulse/steps
    s2 = s0 * np.exp(1j*phase)
    dielectric.plot(ax=ax2, imaginary_axis=ax2twin, alpha=0.1, color=color)
    dielectric.record(s0, s2)
    ax1.plot(dielectric.z[:-1], dielectric.recent_energy[:-1], label="pulse {}".format(pulse + 1), alpha=0.5, c=color)
    ax1.set_title("interference patterns")
    ax1.legend()
    dielectric.reflect(s0).plot(ax3, spectrum_axis=ax4, alpha=0.5, color=color)
    ax3.set_title("reflected")
    ax4.set_title("reflected")
        
plt.tight_layout()        
plt.show()

### Chirps example

In [ ]:
# TODO what is the right unit for the chirp?

z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
probing_wave = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
for chirp in np.linspace(0, 0.002*c.NANO, 3):
    chirped_wave = w.ChirpedPlanarWave(mean=c.GREEN, std=(5*c.MICRO), skew=chirp)
    dielectric.record(chirped_wave, chirped_wave)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    chirped_wave.plot_amplitude(z, ax=ax1)
    ax1.set_title("interference")
    dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
    dielectric.reflect(probing_wave).plot(ax3, spectrum_axis=ax3.twinx())
    plt.tight_layout()
    plt.show()

In [ ]:
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
probing_wave = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
for chirp in np.linspace(0, 0.002*c.NANO, 3):
    chirped_wave = w.ChirpedPlanarWave(mean=c.GREEN, std=(5*c.MICRO), skew=chirp)
    dielectric.record(chirped_wave, probing_wave)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    dielectric.plot_recent_energy(ax1)
    ax1.set_title("interference")
    dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
    dielectric.reflect(probing_wave).plot(ax3, spectrum_axis=ax3.twinx())
    plt.tight_layout()
    plt.show()